<a href="https://colab.research.google.com/github/wynniegross1/vegetation_anomalies/blob/main/ndvi_anomaly_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Calculating NDVI Anomalies for a Specified Region of Interest**


---


Original GEE Script Author: Aaron Eubank

Python Script Adaptation: Wynnie Gross


---


*   This script is run primarily to get a map of the NDVI anomalies compared to a baseline date range.
* This script defines a function to calculate this for a given region of interest.   
* The default dataset is MODIS 8-day product, however it is parameterizable to choose a different dataset.
*   Methodology adapted from GEARS - Geospatial Ecology and Remote Sensing - https://www.geospatialecology.com

In [ ]:
# IMPORT NECESSARY PACKAGES #
import geemap
import ee
from google.colab import userdata
from datetime import datetime

# INITIALIZE GEE PROJECT #
ee.Authenticate()
ee.Initialize(project=userdata.get('projectname'))

####################################
###### NDVI ANOMALY FUNCTION #######
####################################

def calculate_ndvi_anomaly(
    region_of_interest,
    start_date_baseline,
    end_date_baseline,
    start_date_recent,
    end_date_recent,
    source_data='MODIS/061/MOD09A1',
    output_type='percentage',  # Can be 'percentage' or 'raw'
    mask_cropland=True,
    cloud_masking=True
):
    """
    Calculates NDVI anomalies for a given region and time period,
    compared to a baseline period.

    Args:
        region_of_interest: ee.FeatureCollection.  The area to analyze.
        start_date_baseline (str): Start date for the baseline period (YYYY-MM-DD).
        end_date_baseline (str): End date for the baseline period (YYYY-MM-DD).
        start_date_recent (str): Start date for the recent period (YYYY-MM-DD).
        end_date_recent (str): End date for the recent period (YYYY-MM-DD).
        source_data (str):  The Earth Engine ImageCollection ID.  Defaults to MODIS.
        output_type (str): 'percentage' for percentage anomaly, 'raw' for raw difference.
        mask_cropland (bool): Whether to apply a cropland mask.
        cloud_masking (bool):  Whether to apply cloud masking.

    Returns:
        tuple: (ee.Image, geemap.Map) The mean NDVI anomaly image and the map object.
    """

    # Load the image collection
    collection = ee.ImageCollection(source_data)

    # Filter by region of interest
    collection = collection.filterBounds(region_of_interest)

    # Cloud Mask
    def mask_clouds(image):
        """Masks clouds using the StateQA band of MODIS."""
        QA = image.select(['StateQA'])
        bitMask = 1 << 10
        return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))

    # Crop Mask
    def crop_mask(image):
        """Masks non-cropland areas using ESA WorldCover."""
        esa = ee.ImageCollection('ESA/WorldCover/v100')
        esa_latest = esa.limit(1, 'system:time_start').first()
        cropland = esa_latest.updateMask(
            esa_latest.select('Map').eq(40).clip(region_of_interest)
        )
        return image.updateMask(cropland)

    # Function to add NDVI as a band
    def add_ndvi(image):
        """Calculates NDVI and adds it as a band."""
        ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).multiply(
            10000).toInt16()
        return image.addBands(ndvi.rename('ndvi'))

    # Get data for baseline date range
    baseline_collection = collection.filterDate(
        start_date_baseline, end_date_baseline
    ).filter(ee.Filter.calendarRange(121, 303, 'day_of_year'))  #May 1st - Oct 30th

    if cloud_masking:
        baseline_collection = baseline_collection.map(mask_clouds)
    baseline_collection = baseline_collection.map(add_ndvi).select('ndvi')

    baseline_mean = baseline_collection.mean().clip(region_of_interest)

    if mask_cropland:
        baseline_mean = baseline_mean.updateMask(crop_mask(baseline_mean))

    # Get data for recent date range
    recent_collection = collection.filterDate(
        start_date_recent, end_date_recent
    ).filter(ee.Filter.calendarRange(121, 303, 'day_of_year')) #May 1st - Oct 30th

    if cloud_masking:
        recent_collection = recent_collection.map(mask_clouds)
    recent_collection = recent_collection.map(add_ndvi).select('ndvi')

    if mask_cropland:
        recent_collection = recent_collection.map(crop_mask)

    # Subtract the baseline mean from the recent image
    def subtract_mean(image):
        return image.subtract(baseline_mean).copyProperties(
            image, ['system:time_start'])

    # Convert to a percentage anomaly
    def to_percentage(image):
        return image.divide(baseline_mean).multiply(100).copyProperties(
            image, ['system:time_start'])

    anomalies = recent_collection.map(subtract_mean)

    if output_type == 'percentage':
        anomalies = anomalies.map(to_percentage)

    # Find average anomaly
    mean_anomaly = anomalies.mean()

    # Create a map object and automatically center on centroid of ROI
    Map = geemap.Map(center=region_of_interest.geometry().centroid().getInfo()['coordinates'][::-1], zoom=7)

    # Visualization parameters
    veganoms_vis = {
        'min': -50,
        'max': 50,
        'palette': ['darkred', 'red', 'yellow', 'green', 'darkgreen']
    }

    # Add the anomaly layer to the map
    Map.addLayer(mean_anomaly, veganoms_vis, f'Mean NDVI Anomaly ({output_type})')

    # Adding Outline to the Map
    outline = region_of_interest.style(fillColor='00000000')
    Map.addLayer(outline, {}, "Region of Interest")

    # Add legend
    Map.add_colorbar(
        veganoms_vis, label=f"Vegetation Index Anomaly {output_type.capitalize()}",
        layer_name="Mean Anomaly", orientation="horizontal"
    )

    return mean_anomaly, Map


#######################################
########### EXAMPLE USAGE #############
#######################################

# Define Region of Interest (replace with your actual ROI)
region_of_interest = ee.FeatureCollection("projects/ee-aeubank/assets/sudan_admin1").filter(ee.Filter.Or(
    ee.Filter.eq('admin_1', 'Sennar'),
    ee.Filter.eq('admin_1', 'Aj Jazirah')
))

# Set Date Parameters
baseline_start = '2013-01-01'
baseline_end = '2022-12-31'
recent_start = '2023-01-01'
recent_end = datetime.now().isoformat()

# Calculate Anomaly and get map
anomaly_result, anomaly_map = calculate_ndvi_anomaly(
    region_of_interest,
    baseline_start,
    baseline_end,
    recent_start,
    recent_end,
    output_type='percentage'
)

# Display the map
anomaly_map


Map(center=[13.593742636930632, 33.73151043658074], controls=(WidgetControl(options=['position', 'transparent_…